In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [6]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    
    movie = dict(movie) #create a non-destructive copy
    
    alt_titles = {}
    
    # combine alternate titles into one list
    
    for key in ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French',
               
               'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',
               'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish',
               'Revised Romanization', 'Romanized', 'Russian',
               'Simplified', 'Traditional', 'Yiddish']:
        
        if key in movie:

            alt_titles[key] = movie[key]

            movie.pop(key)
            
    if len(alt_titles) > 0:
            
            movie['alt_titles'] = alt_titles
            
        

    # merge column names
    def change_column_name(old_name, new_name):

        if old_name in movie:

            movie[new_name] = movie.pop(old_name)

    change_column_name('Directed by', 'Director')
    change_column_name('Adaptation by','Writer(s)')
    change_column_name('Country of origin','Country')
    change_column_name('Distributed by','Distributor')
    change_column_name('Edited by','Editor(s)')
    change_column_name('Length','Running time')
    change_column_name('Orginal Release','Release date')
    change_column_name('Music By','Composer(s)')
    change_column_name('Produced by','Producer(s)')
    change_column_name('Producer','Producer(s)')
    change_column_name('Productioncompanies','Production company[s]')
    change_column_name('Productioncopany','Production company(s)')
    change_column_name('Released','Release Date')
    change_column_name('Screen story by','Writer(s)')
    change_column_name('Screenplay by','Writer(s)')
    change_column_name('Story by','Writer(s)')
    change_column_name('Theme music composer','Composer(s)')
    change_column_name('Written by','Writer(s)')

    
    return movie

In [30]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory = False)
    
    ratings = pd.read_csv(f'{file_dir}/ratings.csv')


    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        
        
        wiki_movies_raw = json.load(file)
    
    # Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
                  if ('Director' in movie or 'Directed by' in movie)
                  and 'imdb_link' in movie
                  and 'No. of episodes' not in movie]


    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except Exception as e:
        print(e)

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column.
    
    box_office = wiki_movies_df['Box office'].dropna()

    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    
    box_office[box_office.map(lambda x: type(x) != str)]
    
    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\d+\.?\d*\s*[mb]illion'
   
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1.3}(?:,\d{3})+'

    # Add the parse_dollars function.
    def parse_dollars(s):
        
        # if s is not a string return NaN
        if type(s) != str:
            
            return np.nan
        
        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            
            #remove dollar sign and million
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            
            #convert to float and multiply by a million
            value = float(s) * 10**6
            
            #return value
            return value
        
        # in input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            
            # remove the dollar sign and billion
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            
            # convert to float and multiply by a billion
            value = float(s) * 10**9
            
            # return val;ue
            return value
        
        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            
            # remove dollar sign and commas
            s = re.sub('\$|,','',s)
            
            # convert to float
            value = float(s)
            
            # return value
            return value
        
        #otherwise, return NaN
        else:
            
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE) \
        [0].apply(parse_dollars)

    # Clean the budget column in the wiki_movies_df DataFrame.
    
        # drop null values
    budget = wiki_movies_df['Budget'].dropna()
        # convert listing to strings
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
        #remove any values between an dollar sign and a hyphen
    budget = budget.str.replace(r'\$.*[---](?![a-z])', '$', regex=True)
        #remove citation references
    budget = budget.str.replace(r'\[\d+\]\s*', '')
        # parse budget values
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE) \
        [0].apply(parse_dollars)
    

    # Clean the release date column in the wiki_movies_df DataFrame.
    
        #drop null values and make a variable
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
        #make forms
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\s{4}'
    date_form_four = r'\d{4}'
        # extract dates and parse
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    

    # Clean the running time column in the wiki_movies_df DataFrame.
    
        # parse running time
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
        # extract data
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors = 'coerce')).fillna(0)
        # apply function
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 +row[1] if row[2] == 0 else row[2], axis = 1)
    
    
     
    # 2. Clean the Kaggle metadata.
    
        # remove bad data
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis = 'columns')
    
        # convert data types
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors = 'raise')
    
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors = 'raise')
    
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # 3. Merged the two DataFrames into the movies DataFrame.
    
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on = 'imdb_id', suffixes = ['_wiki', '_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    
    movies_df.drop(columns = ['title_wiki', 'release_date_wiki', 'Language', 'Productioncompany '], inplace = True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column],
            
            axis = 1)
        df.drop(columns = wiki_column, inplace = True)


    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')


    # 7. Filter the movies DataFrame for specific columns.
    
    movies_df = movies_df.loc[:, ['imdb_id', 'id', 'title_kaggle', 'original_title', 'tagline', 
                                  'belongs_to_collection', 'url', 'imdb_link', 'runtime', 'budget_kaggle', 'revenue',
                                 'release_date_kaggle', 'popularity', 'vote_average', 'vote_count', 'genres',
                                 'original_language', 'overview', 'spoken_languages', 'Country', 'production_companies',
                                 'production_countries', 'Distributor', 'Producer(s)', 'Director', 'Starring', 'Cinematography',
                                 'Editor(s)', 'Writer(s)', 'Music by', 'Based on']]


    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id',
                     'title_kaggle':'title',
                     'url':'wikipedia_url',
                     'budget_kaggle':'budget',
                     'release_date_kaggle':'release_date',
                     'Country':'country',
                     'Distributor':'distributor',
                     'Producer(s)':'producers',
                     'Director':'director',
                     'Starring':'starring',
                     'Cinematography':'cinematography',
                     'Editor(s)':'editors',
                     'Writer(s)':'writers',
                     'Music by':'composers',
                     'Based on':'based_on'},
                    axis = 'columns', inplace = True)


    # 9. Transform and merge the ratings DataFrame.
        # setting unix
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
        # grouping data
    rating_counts = ratings.groupby(['movieId', 'rating'], as_index = False).count() \
                    .rename({'userId' : 'count'}, axis=1) \
                    .pivot(index = 'movieId', columns = 'rating', values = 'count')
        # rename columns
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
        # merge dataframes
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on = 'kaggle_id', right_index = True, how = 'left')
        # fill in blanks
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
                                 
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [31]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '/Users/user/Desktop/UCSDBootcamp/ETL'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [32]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_43114/1127149120.py:123: FutureWarning: The default value of regex will change from True to False in a future version.
  budget = budget.str.replace(r'\[\d+\]\s*', '')


In [33]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [34]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Music by,Cinematography,Productioncompany,Release date,...,Distributor,Editor(s),Producer(s),Writer(s),Productioncompanies,imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","[Cliff Eidelman, Yello]",Oliver Wood,Silver Pictures,"[July 11, 1990, (, 1990-07-11, )]",...,20th Century Fox,Michael Tronick,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe]",NaN,tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Maurice Jarre,Mark Plummer,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",...,Avenue Pictures,Howard E. Smith,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Charles Gross,Roger Deakins,"[Carolco Pictures, IndieProd Company]","[August 10, 1990, (, 1990-08-10, )]",...,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,"[John Eskow, Richard Rush]",NaN,tt0099005,NaN,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,NaN,"[December 25, 1990, (, 1990-12-25, )]",...,Orion Pictures,Susan E. Morse,Robert Greenhut,Woody Allen,NaN,tt0099012,NaN,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Maurice Jarre,Russell Boyd,NaN,"December 19, 1990",...,Paramount Pictures,David Stiven,John Cornell,Paul Hogan,NaN,tt0099018,NaN,25000000.0,1990-12-19,95.0


In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()